<a href="https://colab.research.google.com/github/2003Yash/AI-powered-EMail-Analyzer/blob/main/easeworkai_ai_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import google.generativeai as ai
import shelve
from faker import Faker


ai.configure(api_key="API_KEY")
model = ai.GenerativeModel("gemini-pro")
chat = model.start_chat()

# Setup local NoSQL storage using shelve
db_path = "email_database"

def store_emails_shelve(n=10):
    fake = Faker()
    with shelve.open(db_path) as db:
        for i in range(n):
            email = {
                "sender": fake.email(),
                "receiver": fake.email(),
                "subject": fake.sentence(),
                "body": fake.paragraph(),
                "timestamp": str(fake.date_time_this_year())
            }
            db[str(i)] = email
    return f"Inserted {n} fake emails into the database."

def get_emails():
    emails = []
    with shelve.open(db_path) as db:
        for key in list(db.keys())[:5]:
            email = db[key]
            emails.append(f"From: {email['sender']}, Subject: {email['subject']}")
    return emails

# Streamlit UI
st.title("📧 Email Assistant Chatbot")

if st.button("Generate Fake Emails"):
    message = store_emails_shelve(10)
    st.success(message)

st.subheader("Recent Emails")
emails = get_emails()
for email in emails:
    st.write(email)

st.subheader("Chat with Email Assistant")
user_input = st.text_input("You:")
if st.button("Send"):
    if user_input:
        prompt = f"""
        You are an intelligent Email Assistant chatbot that understands and helps with various email-related tasks.
        You can assist with:
        1) Sending emails
        2) Querying past emails
        3) Reading emails
        4) Deleting emails
        5) Forwarding emails
        6) Replying to emails

        Emails stored in the database:
        {get_emails()}

        Always provide clear and helpful responses regarding email management.

        User Input: {user_input}
        """
        response = chat.send_message(prompt)
        st.text_area("Bot:", response.text, height=200)
